In [1]:
import pandas as pd
import numpy as np
import datetime

# Create the full data table

In [2]:
data_sets = ["data/parsed/2017-10-11.csv.gz",
            "data/parsed/2017-10-12.csv.gz",
            "data/parsed/2017-10-13.csv.gz",
            "data/parsed/2017-10-14.csv.gz",
            "data/parsed/2017-10-15.csv.gz",
            "data/parsed/2017-10-16.csv.gz",
            "data/parsed/2017-10-17.csv.gz",
            "data/parsed/2017-10-18.csv.gz",
            "data/parsed/2017-10-19.csv.gz",
            "data/parsed/2017-10-20.csv.gz",
            "data/parsed/2017-10-21.csv.gz",
            "data/parsed/2017-10-22.csv.gz",
            "data/parsed/2017-10-23.csv.gz"]

full_data = pd.DataFrame()

for file in data_sets:
    partial_data = pd.read_csv(file)
    full_data = pd.concat([full_data, partial_data], axis=0)
    
full_data.shape

(11440778, 19)

# Remove unnecessary columns

In [3]:
#drop the obviously useless columns
full_data = full_data.drop(['dir'], axis=1)
full_data = full_data.drop(['tst'], axis=1)
full_data = full_data.drop(['hdg'], axis=1)
full_data = full_data.drop(['odo'], axis=1)
full_data = full_data.drop(['oday'], axis=1)
full_data = full_data.drop(['stop_index'], axis=1)
full_data = full_data.drop(['source'], axis=1)
full_data = full_data.drop(['jrn'], axis=1)
full_data = full_data.drop(['line'], axis=1)
full_data = full_data.drop(['oper'], axis=1)
full_data = full_data.drop(['start'], axis=1)
full_data = full_data.drop(['timestamp'], axis=1)

In [4]:
# Give columns more meaningful labels
full_data.rename(columns = {'desi':'line'}, inplace = True)
full_data.rename(columns = {'dl':'delay'}, inplace = True)
full_data.rename(columns = {'spd':'speed'}, inplace = True)
full_data.rename(columns = {'tsi':'time'}, inplace = True)
full_data.rename(columns = {'veh':'vehicle'}, inplace = True)
full_data.rename(columns = {'lat':'latitude'}, inplace = True)
full_data.rename(columns = {'long':'longitude'}, inplace = True)

In [5]:
full_data.head()

,line,delay,latitude,longitude,speed,time,vehicle
0,560,120,60.250462,25.009306,0.09,1507754213,0012_01517
1,23,-33,60.202652,24.903437,10.03,1507754213,0012_01612
2,23,39,60.202940,24.907900,0.00,1507754199,02f42add
3,571,50,60.290020,25.006930,0.00,1507754210,037bee32
4,55,92,60.192116,24.978412,0.00,1507754132,046524ec


# Remove missing data

In [6]:
full_data = full_data[full_data.speed.notnull()]

# Weather data

In [7]:
weather = pd.read_csv("data/all_weather.csv")

weather = weather.drop(['icon'], axis=1)
weather = weather.drop(['summary'], axis=1)

weather['precipType'].fillna('none', inplace=True)
weather['precipType'] = weather['precipType'].astype('category')

weather.head()

,apparentTemperature,cloudCover,dewPoint,humidity,ozone,precipIntensity,precipProbability,precipType,pressure,temperature,time,uvIndex,visibility,windBearing,windGust,windSpeed
0,5.81,0.63,7.21,0.96,314.18,0.0076,0.30,rain,993.96,7.77,1507680000,0,9.72,68,11.99,10.93
1,5.98,0.70,7.19,0.96,314.63,0.0076,0.27,rain,993.88,7.78,1507681200,0,9.24,65,10.77,10.06
2,6.16,0.76,7.18,0.96,315.07,0.0051,0.24,rain,993.80,7.78,1507682400,0,8.74,62,9.53,9.22
3,6.43,0.82,7.18,0.96,315.52,0.0076,0.30,rain,993.75,7.77,1507683600,0,8.69,55,8.30,7.89
4,6.28,0.83,7.13,0.96,315.98,0.0051,0.27,rain,993.75,7.72,1507684800,0,8.64,45,7.77,8.29


# Combine weather data

In [8]:
full_data = full_data.sort_values('time')

full_data = pd.merge_asof(full_data, weather, on='time')

In [9]:
# Remove rows with no weather data
full_data = full_data[full_data.apparentTemperature.notnull()]
full_data.head()

,line,delay,latitude,longitude,speed,time,vehicle,apparentTemperature,cloudCover,dewPoint,...,precipIntensity,precipProbability,precipType,pressure,temperature,uvIndex,visibility,windBearing,windGust,windSpeed
6,615,25,60.31797,24.96730,0.0,1507680025,4d63ce12,5.81,0.63,7.21,...,0.0076,0.3,rain,993.96,7.77,0.0,9.72,68.0,11.99,10.93
7,235N,160,60.20462,24.67328,0.0,1507680040,76d53e3e,5.81,0.63,7.21,...,0.0076,0.3,rain,993.96,7.77,0.0,9.72,68.0,11.99,10.93
8,615,525,60.21282,24.94351,0.0,1507680045,0ebf9e12,5.81,0.63,7.21,...,0.0076,0.3,rain,993.96,7.77,0.0,9.72,68.0,11.99,10.93
9,235N,165,60.20332,24.68024,0.0,1507680105,76d53e3e,5.81,0.63,7.21,...,0.0076,0.3,rain,993.96,7.77,0.0,9.72,68.0,11.99,10.93
10,615,535,60.22117,24.94574,0.0,1507680115,0ebf9e12,5.81,0.63,7.21,...,0.0076,0.3,rain,993.96,7.77,0.0,9.72,68.0,11.99,10.93


# Cull the data to only observe certain routes

In [10]:
unique_lines = pd.unique(full_data.line.ravel())

In [11]:
observed_lines = ['550', '731', '55', '83', '94N']

culled_data = full_data.loc[full_data['line'].isin(observed_lines)]

In [12]:
culled_data.shape

(340723, 22)

In [13]:
culled_data.head()

,line,delay,latitude,longitude,speed,time,vehicle,apparentTemperature,cloudCover,dewPoint,...,precipIntensity,precipProbability,precipType,pressure,temperature,uvIndex,visibility,windBearing,windGust,windSpeed
677,550,50,60.16880,24.80383,0.0,1507685150,25cb9fcf,6.28,0.83,7.13,...,0.0051,0.27,rain,993.75,7.72,0.0,8.64,45.0,7.77,8.29
678,550,50,60.16880,24.80383,0.0,1507685150,25cb9fcf,6.28,0.83,7.13,...,0.0051,0.27,rain,993.75,7.72,0.0,8.64,45.0,7.77,8.29
684,550,80,60.20982,25.07738,0.0,1507685180,55a3d4a3,6.28,0.83,7.13,...,0.0051,0.27,rain,993.75,7.72,0.0,8.64,45.0,7.77,8.29
685,550,80,60.20982,25.07738,0.0,1507685180,55a3d4a3,6.28,0.83,7.13,...,0.0051,0.27,rain,993.75,7.72,0.0,8.64,45.0,7.77,8.29
695,550,67,60.17083,24.81079,0.0,1507685227,25cb9fcf,6.28,0.83,7.13,...,0.0051,0.27,rain,993.75,7.72,0.0,8.64,45.0,7.77,8.29


# Late column

In [14]:
#0 = early/on time, 1 = late
late = []

for i,r in culled_data.iterrows():
    if np.float(r.delay) <= 0:
        late.append(0)
    else:
        late.append(1)
        
culled_data['late'] = late

/Users/szweber/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


# Day of week and hour

In [31]:
days = []
hours = []

for i,r in culled_data.iterrows():
    # Monday is 0, Sunday is 6
    day_of_week = datetime.date.fromtimestamp(r.time).weekday()
    days.append(day_of_week)
    
    # 0-24
    hour_of_day = datetime.datetime.fromtimestamp(r.time).strftime('%H')
    hours.append(int(hour_of_day))

culled_data['dayOfWeek'] = days
culled_data['hourOfDay'] = hours

/Users/szweber/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/szweber/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


# Export the final dataset

In [32]:
culled_data.head()

,line,delay,latitude,longitude,speed,time,vehicle,apparentTemperature,cloudCover,dewPoint,...,pressure,temperature,uvIndex,visibility,windBearing,windGust,windSpeed,late,dayOfWeek,hourOfDay
677,550,50,60.16880,24.80383,0.0,1507685150,25cb9fcf,6.28,0.83,7.13,...,993.75,7.72,0.0,8.64,45.0,7.77,8.29,1,2,4
678,550,50,60.16880,24.80383,0.0,1507685150,25cb9fcf,6.28,0.83,7.13,...,993.75,7.72,0.0,8.64,45.0,7.77,8.29,1,2,4
684,550,80,60.20982,25.07738,0.0,1507685180,55a3d4a3,6.28,0.83,7.13,...,993.75,7.72,0.0,8.64,45.0,7.77,8.29,1,2,4
685,550,80,60.20982,25.07738,0.0,1507685180,55a3d4a3,6.28,0.83,7.13,...,993.75,7.72,0.0,8.64,45.0,7.77,8.29,1,2,4
695,550,67,60.17083,24.81079,0.0,1507685227,25cb9fcf,6.28,0.83,7.13,...,993.75,7.72,0.0,8.64,45.0,7.77,8.29,1,2,4


In [33]:
culled_data.to_csv('culled_data.csv.gz', index=False, compression='gzip')